# Retention rates for US Universities

This project is being built from the ground up to be customizable and reproducible.

After importing the libraries we need, we're going to load up a table of instructions containing which
of the 2,500+ attributes we're going to select from the IPEDS database for data mining. This is a lot,
so it seems more reasonable to externally store the data list we'll be drawing from.

Many of these, such as website addresses or mission statements, aren't going to be terribly useful. Some others we'll need to exclude as it is too closely related to retention rate, and we risk overfitting or circular logic ("hey, here's how to raise your retention rate--have more of them graduate!")

At the root, each entry in the JSON file denotes a separate table. Included alongside the table name are instructions on whether all the table should be imported as default or not, which attributes are continuous, which are discrete, and  which are strings, and whether multiple records exist for each primary key. This is all derived from the associated documentation that comes with IPEDS. 

We're going to load in each table (obviously checking that it doesn't exist first), and extract the correct tables from it. 

(Should you want to change what we're measuring, you can change the predictive variable within the JSON file.)

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import json
import wget
import sys
import os
from zipfile import ZipFile as zf

DATA_PATH = "data"

def import_data(filename):
    with open(filename) as file:
        instructions = json.load(file)
        
# does the data directory exist? if no, create it.
    
    if os.path.isdir(DATA_PATH) is False:
        try:  
            os.mkdir(DATA_PATH)
        except OSError:  
            print ("Could not create data folder.")
            raise
            
# start with an empty dataframe to fill, and get the primary key.
# the primary key should be in the first file in the data list

    ourdata = pd.DataFrame()
    pk = instructions["primarykey"]

# loop through each table. 

    for table in instructions["tables"]:
        
# first, check if the file in the table has been downloaded. if not, download it from
# the path given and throw an error if something is wrong.

        filename = table["name"]+instructions["format"]
        filelocation = DATA_PATH+"/"+filename
        csvfile = DATA_PATH+"/"+table["name"]+".csv"

        if os.path.exists(filelocation) is False:
            try:
                wget.download(instructions["url"]+filename, filelocation)
            except Exception as e:
                print("Problem downloading and saving", table["name"], ":", e)
                raise

# next, if they are zip files, unzip them

        if instructions["format"] == ".zip":
            if os.path.exists(str(csvfile).lower()) is False:
                print("Unzipping "+table["name"])
                with zf(filelocation,"r") as zip_ref:
                    zip_ref.extractall(DATA_PATH)

# load each CSV file into a temporary data frame

        transitional_df = pd.read_csv(str(csvfile).lower(), encoding = "ISO-8859-1")

# then, depending on the instructions in the JSON file, include all the headers, or include a selection.

        if table["includeall"]:

# include the whole list, excluding the specific tables (might be an empty list)
# and add the primary key to select

            to_include = list(transitional_df)
            to_exclude = table["exclude"]
            headers = [x for x in to_include if x not in to_exclude]
            headers.append(pk)

# stick these three lists together plus the primary key

        else:
            headers = [pk, *table["strings"], *table["discrete"], *table["continuous"]]

        selected_headers = [x for x in transitional_df.columns if x in headers]

# if it's the first time around, take the first set of data, checking for the primary key existence

        if ourdata.empty:
            if pk not in transitional_df.columns:
                raise KeyError("Primary key "+pk+" not found in first file mentioned")
            ourdata = transitional_df[selected_headers]

# if not, then we need to join on the primary key

# TODO: FIGURE OUT HOW TO JOIN THE OLD AND NEW DATAFRAMES ON THE KEY
            
import_data("ipeds-instructions.json")

KeyError: 'includeall'